In [1]:
# imports
import os
import glob
import pandas as pd
import random as random
import numpy as np
import obspy
from obspy.core import read
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import warnings
warnings.filterwarnings("ignore")

In [2]:
signalmodel = load_model('signalmodelCNN_v3')
locmodel = load_model('locmodelCNN_v3')

2022-03-14 10:49:14.334636: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-14 10:49:14.334958: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1


In [3]:
def predict(trace, p_prob, s_prob):
    
    ## Things to Specify -- Ensure that they align with trained model
    # For rolling averages
    roll_short = 25
    roll_long = 50

    # For location segments
    window_step = 20
    window_size = 20
    
    # Step size for CNN
    n_steps_sig = 3
    n_steps_loc = 2
    
    # Probability cut-offs
    p_prob = p_prob # for p-wave
    s_prob = s_prob # for signal
    
    ## Reading in Trace and Splitting Channels
    sig_trace = trace.normalize()
    
    samp_rate = sig_trace.stats.sampling_rate
    start_time = sig_trace.stats.starttime
    
    sigs = sig_trace.data
    
    sig_df = pd.DataFrame(sigs, columns = ["trace"])
    
    sigfeatures = []
    
    ## Calculating various features needed for the signal model
    tr = sig_df["trace"]
    mag = abs(tr)
    d = {"trace":tr, "magnitude":mag}
    temp_df = pd.DataFrame(data = d)
    
    temp_df["STA"] = temp_df["magnitude"].rolling(roll_short).mean()
    temp_df["LTA"] = temp_df["magnitude"].rolling(roll_long).mean()
    temp_df["RAV"] = temp_df["STA"]/temp_df["LTA"]
    temp_df["STV"] = temp_df["magnitude"].rolling(roll_short).var()
    temp_df["LTV"] = temp_df["magnitude"].rolling(roll_long).var()

    temp_df.dropna(inplace = True)
    sigfeatures.append(temp_df.values)
    
    sigfeatures = np.array(sigfeatures)
    n_timesteps, n_features, n_outputs = sigfeatures.shape[1], sigfeatures.shape[2], 2
    n_length = int(n_timesteps/n_steps_sig)

    sigfeatures1 = sigfeatures.reshape((sigfeatures.shape[0], n_steps_sig, n_length, n_features))
    
    ## Predicting whether or not there is a pwave in the trace
    is_sig = signalmodel.predict(sigfeatures1)[0][1]

    locfeatures = []

    ## If there is a pwave in the trace, continue to find location of pwave
    start_ind = 0
    end_ind = start_ind + window_size

    while end_ind < (1000 - roll_long):
        trwindow = temp_df["trace"].iloc[start_ind:end_ind]
        magwindow = temp_df["magnitude"].iloc[start_ind:end_ind]
        ravwindow = temp_df["RAV"].iloc[start_ind:end_ind]
        stvwindow = temp_df["STV"].iloc[start_ind:end_ind]
        ltvwindow = temp_df["LTV"].iloc[start_ind:end_ind]

        window_data = {"trace": trwindow, "magnitude": magwindow,
                            "RAV": ravwindow, "STV": stvwindow, "LTV": ltvwindow}

        window_df = pd.DataFrame(data = window_data)

        locfeatures.append(window_df.values)

        start_ind += window_step
        end_ind = start_ind + window_size

    locfeatures = np.array(locfeatures)
    n_timesteps, n_features, n_outputs = locfeatures.shape[1], locfeatures.shape[2], 2
    n_length = int(n_timesteps/n_steps_loc)
    
    locfeatures1 = locfeatures.reshape((locfeatures.shape[0], n_steps_loc, n_length, n_features))
    
    
    prob_vec = locmodel.predict(locfeatures1)[:,1]
    
    # Since we know if there is a p-wave, it will be in the last 3 seconds:
    end_ind = len(prob_vec)
    beg_ind = len(prob_vec) - int(3*samp_rate/window_size + 1)
    p_segment = beg_ind + np.where(prob_vec[beg_ind:end_ind] == max(prob_vec[beg_ind:end_ind]))[0][0]
        
    tick_delta = p_segment*window_step + 0.5*window_size + (roll_long - 1)
        
    time_delta = (tick_delta)/samp_rate
    p_time = start_time + time_delta
        
    if (is_sig >= s_prob)|(max(prob_vec[beg_ind:end_ind]) > p_prob):
        return p_time
    else:
        return None

In [5]:
# get all filenames
datapath = "validation_data"
valid_filenames = [f for f in glob.glob(datapath + "/*/*")]
number_of_files = len(valid_filenames)
number_of_noise = len([f for f in valid_filenames if "_N" in f])
number_of_pwave = number_of_files-number_of_noise

In [6]:
def f1_samples(valid_filenames, p_prob, s_prob, seed):
    
    random.seed(seed)
    # set variables
    accuracy = .5 # max time difference in seconds
    p_win = (.5, 3) # setting limits for P-wave 
    sampling_rate = 31.25
    min_snr = 1.5
    hit = 0; miss = 0; false_alarm = 0; correct_rejection = 0
    hit_accuracy = []
    
    for f in valid_filenames:

        # read in the file
        st = read(f)
        cut = int((random.random()*p_win[1]+p_win[0])*sampling_rate)

        # iterate over traces in mseed files
        for tr in st:

            # there was a comment raised during one of the sessions that some p-waves have very low signal-to-noise ratio
            # here I require all p-waves to have snr 1.5 or greater
            # snr is calcuated as a ratio of sums of abs values of waveforms 1s before and 1s after p-wave arrival
            snr = sum(abs(tr.data[1000:1032]))/sum(abs(tr.data[1000-32:1000]))

            # in case of small snr, just print a message
            if "_P" in f and snr<=min_snr:
                abc = 1
                #print("  Channel: {}, passing, not high-enough P wave S/N ratio ({:4.2f})".format(tr.stats.channel, snr))      

            # otherwise do everything
            else:
                tr.data = tr.data[cut:1000+cut]

                # set correct answers
                if "_N" in f:
                    # if it is a noise segment, the correct answer is None
                    correct_answer = None
                else:
                    # if it is a p-wave segment, the correct answer is the p-wave arrival time (UTCDateTime format)
                    correct_answer = tr.stats.starttime + (1000-cut)/tr.stats.sampling_rate

                prediction = predict(tr, p_prob, s_prob)

                # Evaluation logic
                # noise segment, no p-wave detected
                if prediction is None and correct_answer is None:
                    correct_rejection += 1

                # p-wave segment but no detection
                elif prediction is None and correct_answer is not None:
                    miss += 1

                # p-wave falsely detected in noise segment
                elif prediction is not None and correct_answer is None:
                    false_alarm += 1

                # p-wave detected in p-wave segment
                else:
                    # evaluate if the detection is accurate enough, if so it is a hit...
                    if abs(correct_answer-prediction)<=accuracy:
                        hit += 1
                        hit_accuracy.append(correct_answer-prediction)

                    # ...else it is unfortunately a miss.
                    else:
                        miss += 1
                        hit_accuracy.append(correct_answer-prediction)

    # calculate the stats
    precision = hit/(hit+false_alarm)
    recall = hit/(hit+miss)
    f1score = (2*precision*recall)/(precision+recall)
    
    return(f1score)
    
    

In [9]:
#seed_vec = np.linspace(1, 10, 10)
seed_vec = [1, 5, 10]
p_probs = np.linspace(0.1, 0.2, 6)

In [ ]:
s_prob = 0.1
for p_prob in p_probs:
    seed_f1 = []
    for seed in seed_vec:
        f1 = f1_samples(valid_filenames, p_prob, s_prob, seed)
        seed_f1.append(f1)
        
    avg_f1 = np.mean(seed_f1)
        
    print("P-S", p_prob, "-", s_prob, ":", avg_f1)
            

P-S 0.1 - 0.1 : 0.892388316617
P-S 0.12 - 0.1 : 0.900882375344
P-S 0.14 - 0.1 : 0.903743433941
P-S 0.16 - 0.1 : 0.906532653442
P-S 0.18 - 0.1 : 0.911210757298


In [ ]:
p_probs = np.linspace(0.2, 0.3, 6)

In [ ]:
for p_prob in p_probs:
    seed_f1 = []
    for seed in seed_vec:
        f1 = f1_samples(valid_filenames, p_prob, s_prob, seed)
        seed_f1.append(f1)
        
    avg_f1 = np.mean(seed_f1)
        
    print("P-S", p_prob, "-", s_prob, ":", avg_f1)